# <center>Defining<font color=slate> Context Managers</font></center>
## <center>`context manager`</center>
an object designed to be used in a`with`statement

<h2>

```
with context-manager:
    enter()
    body
    exit()
```
</h2>

A <font color=mediumTurquoise>context-manager</font> ensures tha <font color=mediumTurquoise>resources</font> are properly and automatically <font color=mediumTurquoise>managed</font>

-   `enter()` prepares the manager for use
-   `exit()` cleans it up


In [ ]:
with open(file='important_data.txt', mode='w') as f:
    f.write('The secret password is 12345')

The benefit of using files in a with statement is that they are <font color=mediumTurquoise>automatically</font> closed at the end of the with block.

This works because files are`context managers`.

They have methods which are called by the with statement <font color=mediumTurquoise>before</font> the block has started and <font color=mediumTurquoise>after</font> the block exits.

The `exit` method for a file, that is the code executed <font color=mediumTurquoise>after</font> the with block exits, does the work of closing the file, and this is how files work with with statements to ensure <font color=mediumTurquoise>proper resource management</font>.

File's`exit()`code closes the file
## <center><font color=tomato>Context-Manager </font>Protocol</center>
## `__enter__(self)`
## `__exit__(self, exc_type, exc_val, exc_tb)`

1.  What`with`statement does is execute its expression, the code immediately following the`with`keyword.
The expression must evaluate to a`context manager`. That is the expression must produce an object, which supports both the`__enter__()`and`__exit__()`methods.
2.  Once the expression is evaluated and we have a context manager object, the`with`statement then calls`__enter__()`on that context manager with no arguments.
3.  If`__enter__()`throws an exception, execution never enters the with block, and the `with` statement is done.
4.  If`__enter__()`executes successfully, it can return a value. If the`with`statement includes an `as` clause, this returned value is bound to the name in the as clause.
Otherwise this return value is discarded.
5.  Once `__enter__()` has been executed and its return value potentially bound to a name, the `with` block itself is executed.
The with block can terminate in one of two fundamental ways, with an exception or by running off the end of the block what we call normal termination.
6.  In both cases, the context manager's`__exit__()`method is called after the block. If the block exits normally,`__exit__()`is called with no extra information.
If, on the other hand, the block exits exceptionally, then the exception information is passed to`__exit__()`. This means that`__exit__()`can do different things depending on how the with block terminates

## <center>A first <font color=tomato>content manager</font></center>


In [ ]:
class LoggingContextManager:
    def __enter__(self):
        print('LoggingContextManager.__enter__()')
        return 'You are in a with-block'
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('LoggingContextManager.__exit__({}, {}, {})'.format(exc_type, exc_val, exc_tb))
        return

with LoggingContextManager() as x:
    print(x)

These arguments in `__exit__() `are all None if the block exits normally

In [ ]:
try:
    with LoggingContextManager() as x:
        raise ValueError('Something has gone wrong')
        print(x)

except:
    pass

## <center>`__enter__()`</center>
-   called before entering `with`-statement body
-   return value bond to `as variable`
-   can return value of any type
-   commonly returns context-manager itself

in the following case:

In [ ]:
with open(file='important_data.txt', mode='r') as f:
    data = f.read()


`file.__enter__()` is returning the`file`object itself, to check:


In [ ]:
f = open(file='a_file', mode='w')
with f as g:
    print(f is g)

## <center>`__exit__()`</center>
Called when the with-statement body exits.

-   When a with block exits without an exception, all three of these arguments are set to none.
-   When it exits exceptionally, these arguments are bound to the exception, which terminated the block

## `__exit__(self, exc_type, exc_val, exc_tb)`
<font color=mediumTurquoise>

-   `exc_type` exception type
-   `exc_object` exception object
-   `exc_tb` exception traceback
</font>

### `__exit__()` can check `type` for `None` to see if an <font color=lightGreen>exception was thrown</font>

In [ ]:
class LoggingContextManager:
    def __enter__(self):
        print('LoggingContextManager.__enter__()')
        return 'You are in a with-block'
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            print('LoggingContextManager.__exit__:'
                  'Normal exit detected.')
        else:
            print('LoggingContextManager.__exit__:'
                  'Exception detected'
                  '\ntype={},\nvalue={},\ntraceback={}'.format(exc_type, exc_val, exc_tb))

with LoggingContextManager():
    pass

In [ ]:
try:
    with LoggingContextManager():
        raise ValueError('Core meltdown imminent')

except:
    pass

### By default `__exit__()` <font color=lightGreen>propagates exceptions</font> thrown form the `with`-statement code block
when an exception is thrown from a with block, the context managers `__exit__()` is executed,
and afterward the original exception is re raised.

In [ ]:
try:
    with LoggingContextManager():
        raise ValueError('Core meltdown imminent')
except ValueError:
    print('*** ValueError detected ***')


-   if`__exit__()`<font color=mediumTurquoise>returns </font>`False`the exception is <font color=mediumTurquoise>propagated</font>.
-   `__exit__()` answers the question "should the `with`-statement swallow exception?"
-   By default functions return`Nonw`. `None` evaluates to `False`
-   `__exit__()` should <font color=mediumTurquoise>never</font> explicitly re-raise exceptions
-   If `__exit__()` wants to ensure that the exception from the `with` block is re raised, it should simply return `False` and let the `with` statement re raise it.
-   `__exit__()` should only <font color=mediumTurquoise>raise</font> exceptions if it <font color=mediumTurquoise>fails</font> itself

## <center>`contextlib`<font color=tomato><br>standard library module for working with context managers</font></center>
Provides utilities for common tasks involving the `with` statement

## contextlib.contextmanager -> contextmanager
A decorator to create new context managers

In [ ]:
import contextlib

@contextlib.contextmanager
def my_context_manager():
    # <ENTER>
    try:
        yield ['value']
        # <NORMAL EXIT>
    except:
        # <EXCEPTIONAL EXIT>
        raise
with my_context_manager() as x:
    pass


Rewriting our `LoggingContextManager` with the `contextmanager` it would be as follows:

In [48]:
import contextlib

@contextlib.contextmanager
def logging_context_manager():
    print('logging_context_manager: enter')
    try:
        yield 'You are in a with-block'
        print('logging_context_manager: normal exit')
    except:
        print('logging_context_manager: exceptional exit')

with logging_context_manager() as x:
    print(x)

logging_context_manager: enter
You are in a with-block
logging_context_manager: normal exit


In [49]:
    with logging_context_manager() as x:
        raise ValueError('Something has gone wrong')

logging_context_manager: enter
logging_context_manager: exceptional exit


-   `contextmanager` uses standard exception to propagate exceptions
-   `contextmanager` explicitly re-raise or so not catch to propagate exceptions
-   `contextmanager` swallow exceptions by no re-raising them

let's update our new context manager to propagate exceptions. We simply add a bare <font color=mediumTurquoise>raise</font> call after logging the exception.


In [52]:
import contextlib

@contextlib.contextmanager
def logging_context_manager():
    print('logging_context_manager: enter')
    try:
        yield 'You are in a with-block'
        print('logging_context_manager: normal exit')
    except:
        print('logging_context_manager: exceptional exit')
        raise


In [55]:
try:
        with logging_context_manager() as x:
            raise ValueError('Something has gone wrong')
except Exception as e:
    print(e.__repr__())

logging_context_manager: enter
logging_context_manager: exceptional exit
ValueError('Something has gone wrong')


## <center>multiple<font color=tomato> context managers</font></center>
`with`-statements can use as many context-managers as needed

```
with cm1() as a, cm2() as b, ....:
    BODY```

it works as nested `with` statements:

is the same as:

example:

In [56]:
import contextlib

@contextlib.contextmanager
def nest_test(name):
    print('Entering', name)
    yield
    print('Exiting', name)

with nest_test('outer'), nest_test('inner'):
    print('BODY')

Entering outer
Entering inner
BODY
Exiting inner
Exiting outer


with `as variable`


In [ ]:
with nest_test('outer'):
    with nest_test('inner'):
        print('BODY')
import contextlib

In [65]:
@contextlib.contextmanager
def nest_test(name):
    print('Entering', name)
    yield name
    print('Exiting', name)

with nest_test('outer') as n1, nest_test('inner, nested in ' + n1):
    print('BODY')

Entering outer
Entering inner, nested in outer
BODY
Exiting inner, nested in outer
Exiting outer


Exceptions propagated from <font color=mediumTurquoise>inner</font> context managers will be seen by <font color=mediumTurquoise>outer</font> context managers

In [66]:
import contextlib

@contextlib.contextmanager
def propagater(name, propagate):
    try:
        yield
        print(name, 'exited normally.')
    except Exception:
        print(name, 'received an exception!')
        if propagate:
            raise

inner received an exception!
outer exited normally.


Inner context manager swallows the exception so that the outer one never sees it

In [ ]:
with propagater('outer', True), propagater('inner', False):
    raise TypeError('Cannot convert lead into gold')

Inner context manager propagates the exception while the outer one swallows it

In [67]:
with propagater('outer', False), propagater('inner', True):
    raise TypeError('Cannot convert lead into gold')

inner received an exception!
outer received an exception!


## <center>Context Managers for <font color=tomato>transactions: </font> example</center>
This example will involve a Connection class, which represents some sort of database connection, along with the Transaction class, which manages transactions in the database.
Users of our system can create connections and then create transaction objects to start transactions.
To commit or roll back transactions, users can call methods on the transaction instances.

In [68]:

class Connection:
    def __init__(self):
        self.xid = 0

    def _start_transaction(self):
        print('starting transaction', self.xid)
        rslt = self.xid
        self.xid = self.xid + 1
        return rslt

    def _commit_transaction(self, xid):
        print('committing transaction', xid)

    def _rollback_transaction(self, xid):
        print('rolling back transaction', xid)

import contextlib


class Transaction:
    def __init__(self, conn):
        self.conn = conn
        self.xid = conn._start_transaction()

    def commit(self):
        self.conn._commit_transaction(self.xid)

    def rollback(self):
        self.conn._rollback_transaction(self.xid)


@contextlib.contextmanager
def start_transaction(connection):
    tx = Transaction(connection)

    try:
        yield tx
    except Exception:
        tx.rollback()
        raise

    tx.commit()

starting transaction 0
rolling back transaction 0
Ooops! Transaction 0 failed.


In [72]:
conn = Connection()
try:
    with start_transaction(conn) as tx:
        x = 1 + 1
        raise ValueError()
        y = x + 2
        print('transaction 0 = ', x, y)
except ValueError:
    print('Ooops! Transaction 0 failed.')

starting transaction 0
rolling back transaction 0
Ooops! Transaction 0 failed.


In [73]:
try:
    with start_transaction(conn):
        x = 1 + 1
        y = x + 2
        print('transaction 1 = ', x, y)
except ValueError:
    assert False

starting transaction 1
transaction 1 =  2 4
committing transaction 1
